In [1]:
import os 
import torch
import numpy as np
from dataset import dataset_VQ, dataset_TM_eval
from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
from exit.utils import animate3d, t2m_bone
import clip
from utils.motion_process import recover_from_ric
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans_multi as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions
from tqdm import tqdm
from exit.utils import get_model, visualize_2motions, generate_src_mask, init_save_folder, uniform, cosine_schedule
from einops import rearrange, repeat
import torch.nn.functional as F
from exit.utils import base_dir
from models.vqvae_multi import VQVAE_MULTI_V2
from models.vqvae_general import HumanVQVAE_GENERAL,VQVAE_decode_only

In [4]:
class Temp:
    def __init__(self, extra_args=None):
        print('mock:: opt')
        if extra_args is not None:
            for i in extra_args:
                self.__dict__[i] = extra_args[i]
args = Temp()

args.dataname = args.dataset_name = 't2m'
args.nb_code = 256
args.code_dim = 32
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512


args.embed_dim_gpt = 1280
args.block_size = 51
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4

args.total_iter = 10
args.batch_size = 1
args.vq_name ='2024-08-12-23-14-09_vq5_multi'
args.resume_trans='/home/haoyum3/MMM/output/t2m/2024-08-14-12-01-17_trans_multi_1/net_last.pth'
args.resume_pth='/home/haoyum3/MMM/output/vq/2024-08-12-23-14-09_vq5_multi/net_last.pth'
args.num_local_layer =2
args.pkeep = 0.5


mock:: opt


In [5]:
from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 1460/1460 [00:03<00:00, 457.81it/s]

Pointer Pointing at 0


In [14]:
args.teacher_pth="/home/haoyum3/MMM/output/vq/2024-08-16-23-15-13_multi_vq_256_32_5/net_last.pth"
args.vq_act='relu'
args.vq_norm=None
if args.teacher_pth:
    teacher_net= VQVAE_MULTI_V2(args, ## use args to define different parameters in different quantizers
                            args.nb_code,#8192
                            args.code_dim,#32
                            args.output_emb_width,#512
                            args.down_t,#2
                            args.stride_t,#2
                            args.width,#512
                            args.depth,#3
                            args.dilation_growth_rate,#3
                            args.vq_act,#'relu'
                            args.vq_norm,#None
                            {'mean': torch.from_numpy(val_loader.dataset.mean).cuda().float(), 
                            'std': torch.from_numpy(val_loader.dataset.std).cuda().float()},
                            True)
    teacher_ckpt=torch.load(args.teacher_pth, map_location='cpu')
    teacher_net.load_state_dict(teacher_ckpt['net'], strict=True)
    teacher_net.cuda()
    teacher_net.eval()
net= VQVAE_decode_only(args, ## use args to define different parameters in different quantizers
                        teacher_net,
                        args.nb_code,#8192
                        args.code_dim,#32
                        args.output_emb_width,#512
                        args.down_t,#2
                        args.stride_t,#2
                        args.width,#512
                        args.depth,#3
                        args.dilation_growth_rate,#3
                        args.vq_act,#'relu'
                        args.vq_norm,#None
            )
args.resume_pth='/home/haoyum3/MMM/output/vq/2024-08-19-00-17-48_vq_general_decoder/net_last.pth'
if args.resume_pth : 
    ckpt = torch.load(args.resume_pth, map_location='cpu')
    net.load_state_dict(ckpt['net'], strict=True)
#net = torch.nn.DataParallel(net)
net.eval()
net.cuda()

VQVAE_decode_only(
  (decoder): Decoder(
    (model): Sequential(
      (0): Conv1d(160, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): Sequential(
        (0): Resnet1D(
          (model): Sequential(
            (0): ResConv1DBlock(
              (norm1): Identity()
              (norm2): Identity()
              (activation1): ReLU()
              (activation2): ReLU()
              (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
              (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
            )
            (1): ResConv1DBlock(
              (norm1): Identity()
              (norm2): Identity()
              (activation1): ReLU()
              (activation2): ReLU()
              (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
              (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
            )
            (2): ResConv1DBlock(
              (no

In [8]:
args.batch_size=1
args.window_size=64
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        args.batch_size,
                                        window_size=args.window_size,
                                        unit_length=2**args.down_t)

train_loader_iter = dataset_VQ.cycle(train_loader)

  0%|          | 0/23384 [00:00<?, ?it/s]

100%|██████████| 23384/23384 [00:39<00:00, 590.39it/s]

Total number of motions 20942


In [17]:
gt_motion = next(train_loader_iter)
gt_motion = gt_motion.cuda().float() # bs, nb_joints, joints_dim, seq_len
pose = gt_motion[0, :].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
pose = recover_from_ric(torch.from_numpy(pose).float(), 22).numpy()
animate3d(pose, BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)
pred_motion = net(gt_motion)
pose = pred_motion[0, :].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
pose = recover_from_ric(torch.from_numpy(pose).float(), 22).numpy()
animate3d(pose, BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)

In [20]:
gt_motion1 = gt_motion.clone()
tokens=teacher_net(gt_motion1,type='encode')
print(tokens.shape)
gt_motion2 = next(train_loader_iter)
pose = gt_motion2[0, :].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
pose = recover_from_ric(torch.from_numpy(pose).float(), 22).numpy()
animate3d(pose, BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)

torch.Size([1, 16, 5])


In [32]:
gt_motion1 = gt_motion.clone()
tokens1=teacher_net(gt_motion1,type='encode')
print(tokens1.shape)
gt_motion2=gt_motion2.cuda().float()
tokens2=teacher_net(gt_motion2,type='encode')


torch.Size([1, 16, 5])


In [33]:
# tokens1[...,0]=tokens2[...,0]
# tokens1[...,1]=tokens2[...,1]
tokens1[...,2]=tokens2[...,2]
tokens1[...,3]=tokens2[...,3]
# tokens1[...,4]=tokens2[...,4]
emb=teacher_net(tokens1,type='decode')
print(emb.shape)
pose = emb[0, :].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
pose = recover_from_ric(torch.from_numpy(pose).float(), 22).numpy()
animate3d(pose, BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)

torch.Size([1, 64, 263])


In [34]:
# token_emb=teacher_net(emb,type='token_emb')
# print(token_emb.shape)
pred_motion = net(emb)
pose = pred_motion[0, :].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
pose = recover_from_ric(torch.from_numpy(pose).float(), 22).numpy()
animate3d(pose, BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)

In [7]:
import torch
from einops import rearrange, repeat
batch_size = 10
max_len = 50
m_tokens_len = torch.randint(1,50,(batch_size,))
rand_mask_probs = torch.zeros(batch_size).float().uniform_(0.5, 1)
# rand_mask_probs = cosine_schedule(rand_mask_probs)
num_token_masked = (m_tokens_len * rand_mask_probs).round().clamp(min = 1)
batch_randperm = torch.rand((batch_size, max_len))
batch_randperm = batch_randperm.argsort(dim=-1)
mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1')

In [11]:
import torch
batch_size = 10
max_len = 50
m_tokens_len = torch.randint(1,50,(batch_size,))
rand_mask_probs = torch.zeros(batch_size).float().uniform_(0.5, 1)
# rand_mask_probs = cosine_schedule(rand_mask_probs)
num_token_masked = (m_tokens_len * rand_mask_probs).round().clamp(min = 1)
batch_randperm = torch.rand((batch_size, max_len))
batch_randperm = batch_randperm.argsort(dim=-1)
mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1')
mask_token = mask_token.unsqueeze(-1).repeat(1,1,5)
mask_token= mask_token.permute(0,2,1)

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False,  True, False,  True, False,
        False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False, False])
